In [2]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, LSTM, GRU, SimpleRNN, Embedding, Conv1D, MaxPooling1D
from keras import optimizers,regularizers
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

In [2]:
def create_dataset(data, n_steps):
    X_train, Y_train, X_test, Y_test = list(), list(), list(), list()
    final_choices = [x[-1] for x in data]
    features = [x[:-1] for x in data]
    # making training data 
    count = 0
    for feat,f_choice in zip(features, final_choices):
        if(len(feat)//2 <= n_steps):
            X_test.append(feat)
            Y_test.append(f_choice)
            X_train.append(feat)
            Y_train.append(f_choice)
            count += 1
        else:
            for i in range(len(feat)//2-n_steps):
                end_idx = 2*(i+n_steps)
                X_train.append(feat[2*i:end_idx])
                Y_train.append(feat[end_idx])
            X_test.append(feat[(len(feat)-2*n_steps):len(feat)])
            Y_test.append(f_choice)
    print(count)
    return X_train, Y_train, X_test, Y_test

In [3]:
def normalise_outcomes(data):
    maxi = max([max(x) for x in data])
    mini = min([min(x) for x in data])
    for i in range(len(data)):
        for j in range(1,len(data[i]),2):
            data[i][j] -= mini
            data[i][j] /= (maxi-mini)

In [4]:
def parse_dataset(name, n_steps):
    infile = open(name, 'r')
    lines = infile.readlines()
    data = []
    for line in lines:
        data.append([float(x) for x in line.split(',')])
#     normalise_outcomes(data)
    return create_dataset(data, n_steps)

In [5]:
n_steps=2
X_train, y_train, X_test, y_test = parse_dataset('data/estimation_without_padding.csv',n_steps)

188


In [6]:
y_train = np.array(y_train)
y_test = np.array(y_test)
# y_train -= 1
# y_test -= 1

X_train = pad_sequences(X_train, value=-50, padding='post', dtype=float)
X_test = pad_sequences(X_test, value=-50, padding='post', dtype=float)

In [7]:
X_test[0]

array([ 2. , -0.3,  1. , -0.3])

In [8]:
X_train = np.reshape(X_train, (X_train.shape[0], n_steps, 2))
X_test = np.reshape(X_test, (X_test.shape[0], n_steps, 2))

In [9]:
X_train[0]

array([[ 1. , -0.3],
       [ 2. , -0.3]])

In [10]:
d = 0.2

In [11]:
model = Sequential()
model.add(LSTM(50, return_sequences=False, activation='relu', kernel_initializer='normal', input_shape=(None, X_train.shape[2])))
# model.add(Dense(10))
model.add(Dense(3, activation='softmax'))

In [12]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
X_train.shape, X_test.shape

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50)                10600     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 153       
Total params: 10,753
Trainable params: 10,753
Non-trainable params: 0
_________________________________________________________________


((11250, 2, 2), (1170, 2, 2))

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1)

Train on 11250 samples, validate on 1170 samples
Epoch 1/10
11250/11250 [==============================] - 60s 5ms/step - loss: 0.5406 - acc: 0.7410 - val_loss: 0.7567 - val_acc: 0.6256
Epoch 2/10
11250/11250 [==============================] - 60s 5ms/step - loss: 0.5078 - acc: 0.7648 - val_loss: 0.7378 - val_acc: 0.5624
Epoch 3/10
11250/11250 [==============================] - 59s 5ms/step - loss: 0.5042 - acc: 0.7666 - val_loss: 0.7269 - val_acc: 0.6274
Epoch 4/10
11250/11250 [==============================] - 60s 5ms/step - loss: 0.5031 - acc: 0.7701 - val_loss: 0.7650 - val_acc: 0.5393
Epoch 5/10
11250/11250 [==============================] - 61s 5ms/step - loss: 0.5005 - acc: 0.7711 - val_loss: 0.7211 - val_acc: 0.5632
Epoch 6/10
11250/11250 [==============================] - 58s 5ms/step - loss: 0.4995 - acc: 0.7704 - val_loss: 0.7415 - val_acc: 0.5376
Epoch 7/10
 5560/11250 [=============>................] - ETA: 28s - loss: 0.4971 - acc: 0.7721

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
model.predict(X_test[0:10]), y_train[0:10]

In [ ]:
predictions = model.predict(X_test)

In [ ]:
ans=0
for pred, truth in zip(predictions,y_test):
    if(pred>=0.5 and truth==1):
        ans += 1
    if(pred<0.5 and  truth==0):
        ans += 1
ans*100.0/(len(y_test))

In [ ]:
%%script false
1, 61.45
2, 54.70
3, 54.62
5, 58.38
10, 62.31
20, 60.00

In [ ]:
x=[1,2,3,5,10,20]
y=[61.45, 54.70, 54.62, 58.38, 62.31, 60.00]
plt.plot(x,y, '--bo')